## Analyzing Excel Based Actuarial Models using Knowledge Bases for Amazon Bedrock


In [2]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws --force-reinstall --quiet 
%pip install -U langchain_community --force-reinstall --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.33 which is incompatible.
sagemaker 2.215.0 requires attrs<24,>=23.1.0, but you have attrs 24.2.0 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.33 which is incompatible.

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
kb_id = "5FM0C7LUFJ"

### Follow the steps below to initiate the bedrock client:

1. Import the necessary libraries, along with langchain for bedrock model selection, llama index to store the service context containing the llm and embedding model instances. We will use this service context later in the notebook for evaluating the responses from our Q&A application. 

2. Initialize `anthropic.claude-3-sonnet-20240229-v1:0` as our large language model to perform query completions using the RAG pattern with the given knowledge base, once we get all text chunk searches through the `retrieve` API.

In [5]:
import boto3
import pprint
from botocore.client import Config
import json
from IPython.display import Markdown, display

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [6]:
def retrieve(query, kbId, numberOfResults=20):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### Initialize your Knowledge base id before querying responses from the initialized LLM

Next, we will call the `retreive API`, and pass `knowledge base id`, `number of results` and `query` as paramters. 

`score`: You can view the associated score of each of the text chunk that was returned which depicts its correlation to the query in terms of how closely it matches it.

### Extract the text chunks from the retrieveAPI response

In the cell below, we will fetch the context from the retrieval results.

In [7]:
def get_prompt(prompt_contexts, prompt_query, prompt_excel_tab):
    prompt = f"""
    Human: You are an insurance actuarial model specialist, and provides answers to questions by using fact based and statistical information when possible. 
    You have vast understanding of excel and actuarial models created in excel or other actuarial systems. 
    The context and excel_tab will contain Cell, Values and Formulas. You need to understand the Cell Values and Formulas. 
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    The goal is to identify which cells (a group of cells or individual cells) form variables for the purposes of actuarial model documentation. 
    There are two types of variables. 
    i) Model variable: A model variable has at least one cell with a formula in it. A formula is anything that is not just a number. A model variable can only contain cells from the same tab. Within the same tab, the cells are adjacent (left, right, above, below, diagonal - adjacent
    via a corner).
    ii) Input variable: A variable that only has numbers (not formulas at all), we call an input variable.

    The provided knowledgebase has text files with exported formulas. The extracted text is in <excel_tab> tag. The Following are the rules for parsing the text files. 
    1. All formulas in a single columns represented as a one line.
    2. Adjacent columns are generally part of a variable. 
    3. A variable will have rows and columns next to each other. For e.g. A1, B1, C1, A2, B2, C3, A3, B3, C3 etc. 
    4. If there is a blank column or multiple blank columns then it is a new variable. Hence, identify gaps or blank columns. For e.g. A1, B1, A2, B2, D1, E1, D2, E2 are two variables A:B and D:E  
    4. A text above or just beside are name of the variable
    5. One sheet may refer to formulas in the same sheet. If they refer to other sheet then the sheet name will be appended to the Cell for e.g. 'Sales Plan'!A2. 
    6. If there is a gap or blank row in the variable then its is a new variable. Hence, variable will not have any blank row. 
    7. You need to identify model variables and input variables. 

    The approach to find the model variables will be: 
    1) Identify the islands/tables first
    2) Use knowledge of the islands you’ve found to express the relationships between the islands/tables and variables

    <context>
    {prompt_contexts}
    </context>

    <question>
    {prompt_query}
    </question>

    <excel_tab>
    {prompt_excel_tab}
    </excel_tab>

    The response should be specific and use your insurance knowlegde, actuarial model fundamentals, excel formulae or numbers when possible.

    Assistant:"""
    
    return prompt


In [8]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [9]:
# payload with model paramters
excel_tab_name = "Calculation Engine"

excel_tab_content = ""

query = f"""What are the tables present in the {excel_tab_name} tab"""

response = retrieve(query, kb_id, 25)
retrievalResults = response['retrievalResults']
contexts = get_contexts(retrievalResults)

prompt = get_prompt(contexts, query, excel_tab_content)
messages=[{ "role":'user', "content":[{'type':'text','text': prompt}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "messages": messages,
    "temperature": 0.3,
    "top_p": 1
        }  )

In [10]:
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

display(Markdown(response_text))

Based on the information provided in the context, I can identify several key tables or groups of data present in the Calculation Engine tab:

1. Policy Information Table:
   This table appears to contain basic policy details, including:
   - Policy number (Cell A6)
   - Policyholder issue age (Cell A9)
   - Policyholder sex (Cell A10)
   - Policyholder smoking status (Cell A11)
   - Underwriting loadings (Cell A12)
   - Policy Cover Effective date (Cell A13)
   - Face Value (Cell A14)
   - Annual Premium (Cell A15)
   - Aggregate Premium (Cell A16)
   - Payment term (Cell A17)
   - Frequency (Cell A18)
   - tPx (mortality probability, Cell A19)
   - Policy loan (Cell A20)
   - SP / RP (Single Premium / Regular Premium indicator, Cell A21)
   - Scalar (Cell A22)
   - IF / NB (In Force / New Business indicator, Cell A23)

2. Fund Value / Premium Funding Account Table (Cell A34)

3. Model Points Table:
   This appears to be a more detailed version of the policy information, starting from Cell B8 and including similar fields to the Policy Information Table.

4. Calculation Results Table:
   This seems to be a large table that spans multiple columns (at least from column N to AU) and many rows (at least 993). It likely contains calculated values for various policy metrics over time.

5. Offset Tables:
   There appear to be tables created using the OFFSET function, particularly referencing columns AO and AT. These might be used for time-based calculations or projections.

6. Mortality Table:
   While not explicitly mentioned, the presence of tPx suggests that there's likely a mortality table used in the calculations.

These tables work together to form the core of an actuarial model, likely used for policy valuation, projection, or pricing. The model seems to calculate values on a monthly basis (references to *12 in formulas) and uses various actuarial and financial concepts typical in life insurance modeling.

In [52]:
def truncate_text(text, start, max_words=5120):
    # Split the text into words
    words = text.split()
    
    # If the number of words is less than or equal to max_words, return the original text
    if len(words) <= max_words:
        return text, ""
    
    truncated_words = words[start:start+max_words]
    remaining_words = words[start + max_words:len(words)]
    truncated_text = ' '.join(truncated_words)
    remaining_words = ' '.join(remaining_words)
    
    # Otherwise, join the first max_words and add an ellipsis
    return truncated_text, remaining_words

In [55]:
# payload with model paramters

import os
import time

# Set the directory path
directory_path = "./model_files"

# The following variable will be used for creation of summary and identify all input and output variables. 
Total_Response = "" 

completeFileName = f"""./model_files/{excel_tab_name}_row.txt"""
print(completeFileName)
file = open(completeFileName, "r")
excel_tab_content = file.read()
file.close()


contexts = ""
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
accept = 'application/json'
contentType = 'application/json'

#excel_tab_content = truncate_text(excel_tab_content)

words = excel_tab_content.split()
print("Total number of words in a file are = ", len(words)) 
start = 0
response_text = ""
i = 1 

while len(words) > 0 and i < 3:
    i +=1
    print("Processing chunk ", start)
    # Truncate the text to ensure that prompt is not failing
    excel_tab_content_part, excel_tab_content = truncate_text(excel_tab_content, start)
    start += 5120
    print("Remaining words = ", len(excel_tab_content.split()))
    prompt = get_prompt(contexts, query, excel_tab_content_part)
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "messages": messages,
        "temperature": 0.4,
        "top_p": 1
            }  )

    response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    response_text = response_text + response_body.get('content')[0]['text']
    #response_text += "--"*100
    words = excel_tab_content.split()
    print("Remaining words = ", len(words) )

./model_files/Calculation Engine_row.txt
Total number of words in a file are =  173959
Processing chunk  0
Remaining words =  168839
Remaining words =  168839
Processing chunk  5120
Remaining words =  158599
Remaining words =  158599


In [56]:

display(Markdown(response_text))

Based on the provided Excel tab information, I can identify several key tables present in the Calculation Engine tab:

1. Policy Information Table (columns B-H):
   This table contains basic policy details like policy number, issue age, sex, smoking status, effective date, face value, premium, payment term, and frequency.

2. Demographic Rates Table (columns J-P):
   This table calculates mortality and surrender rates. It includes CSO mortality rates, underwriting mortality rates, and surrender rates.

3. Policy Values Table (columns Q-Y):
   This table tracks the number of policies in force, policy surrender values, and reserves.

4. Cash Flow Table (columns Z-AO):
   This table calculates various cash flows including premiums, investment income, death benefits, surrender benefits, expenses, and overall profit/loss.

5. Discounting Table (columns AQ-AT):
   This table contains spot rates, forward rates, and discount factors used for present value calculations.

6. Sum at Risk Table (column AU):
   This column calculates the sum at risk for each policy.

7. Policy Loan Table (column AW):
   This column appears to track policy loan amounts, though it's noted as "To be refined".

8. Premium Funding Account (PFA) Table (columns AZ-BC):
   This table tracks the PFA balance, including opening balance, premium payments, closing balance, and investment income.

These tables form the core structure of the actuarial model, covering key aspects such as policy details, demographic assumptions, policy values, cash flows, and financial calculations. The model appears to be a comprehensive life insurance projection model, likely used for pricing, valuation, or financial reporting purposes.Based on the provided formulas, there appear to be several key tables or groups of related calculations in the Calculation Engine tab:

1. Policy Information Table (columns E-H):
   - Tracks policy year, duration, and dates

2. Mortality and Lapse Rates Table (columns J-Q):
   - Calculates mortality rates, lapse rates, and persistency

3. Account Value Calculation Table (columns S-W):
   - Computes account value, interest, and related items

4. Premium and Benefit Calculations (columns Y-AC):
   - Calculates premiums, benefits, and related cash flows

5. Expense Calculations (columns AD-AJ):
   - Computes various policy expenses and commissions

6. Cash Flow Summary (columns AL-AO):
   - Summarizes key cash flow components

7. Interest Rate and Discounting Calculations (columns AQ-AT):
   - Calculates interest rates and discount factors

8. Additional Policy Metrics (columns AU-BC):
   - Computes various policy-related metrics and reserves

These tables work together to project the financial performance of insurance policies over time, incorporating factors like mortality, lapses, expenses, and interest rates. The model appears to be a typical actuarial projection model for life insurance or annuity products.

In [57]:
# payload with model paramters

excel_tab_name = "Financial Inputs"

import os
import time

# Set the directory path
directory_path = "./model_files"

# The following variable will be used for creation of summary and identify all input and output variables. 
Total_Response = "" 

completeFileName = f"""./model_files/{excel_tab_name}.txt"""
print(completeFileName)
file = open(completeFileName, "r")
excel_tab_content = file.read()
file.close()


contexts = ""
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
accept = 'application/json'
contentType = 'application/json'

#excel_tab_content = truncate_text(excel_tab_content)

words = excel_tab_content.split()
print("Total number of words in a file are = ", len(words)) 
start = 0
response_text = ""
i = 1 

while len(words) > 0 and i < 3:
    i +=1
    print("Processing chunk ", start)
    # Truncate the text to ensure that prompt is not failing
    excel_tab_content_part, excel_tab_content = truncate_text(excel_tab_content, start)
    start += 5120
    print("Remaining words = ", len(excel_tab_content.split()))
    prompt = get_prompt(contexts, query, excel_tab_content_part)
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "messages": messages,
        "temperature": 0.4,
        "top_p": 1
            }  )

    response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    response_text = response_text + response_body.get('content')[0]['text']
    #response_text += "--"*100
    words = excel_tab_content.split()
    print("Remaining words = ", len(words) )

./model_files/Financial Inputs.txt
Total number of words in a file are =  294
Processing chunk  0
Remaining words =  0
Remaining words =  0


In [58]:

display(Markdown(response_text))

Based on the provided information in the Excel tab "Financial Inputs," I can identify the following tables present in this sheet:

1. Balance Sheet Table:
   This table appears to start at cell E3 and includes the following components:
   - Assets (E3-E7)
   - Liabilities (E9-E10)

2. Profit & Loss Table:
   This table starts at cell E12 and includes:
   - Income (E13-E15)
   - Outgo (E16-E21)

3. Other Table:
   This small table starts at cell E24 and includes:
   - Projected Actual Expenses (E25)

4. Financial Statement Structure Table:
   This table appears to be in columns A-C, rows 3-21, providing a hierarchical structure of financial statement items (FS_BS Lvl 1, FS_BS Lvl 2, FS_BS Lvl 3).

5. Year Projection Table:
   This table is in row 2, columns H-K, showing a simple year progression formula (each cell is the previous year plus one).

It's important to note that the question asks about tables in the "Calculation Engine" tab, but the provided Excel tab is named "Financial Inputs." Therefore, I cannot definitively answer about tables in the Calculation Engine tab. The information I've provided is based on the Financial Inputs tab data given in the <excel_tab> section.

In [59]:
excel_tab_name = "Calculation Model Points"

import os
import time

# Set the directory path
directory_path = "./model_files"

# The following variable will be used for creation of summary and identify all input and output variables. 
Total_Response = "" 

completeFileName = f"""./model_files/{excel_tab_name}.txt"""
print(completeFileName)
file = open(completeFileName, "r")
excel_tab_content = file.read()
file.close()


contexts = ""
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
accept = 'application/json'
contentType = 'application/json'

#excel_tab_content = truncate_text(excel_tab_content)

words = excel_tab_content.split()
print("Total number of words in a file are = ", len(words)) 
start = 0
response_text = ""
i = 1 

while len(words) > 0 and i < 3:
    i +=1
    print("Processing chunk ", start)
    # Truncate the text to ensure that prompt is not failing
    excel_tab_content_part, excel_tab_content = truncate_text(excel_tab_content, start)
    start += 5120
    print("Remaining words = ", len(excel_tab_content.split()))
    prompt = get_prompt(contexts, query, excel_tab_content_part)
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "messages": messages,
        "temperature": 0.4,
        "top_p": 1
            }  )

    response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    response_text = response_text + response_body.get('content')[0]['text']
    #response_text += "--"*100
    words = excel_tab_content.split()
    print("Remaining words = ", len(words) )
    
display(Markdown(response_text))

./model_files/Calculation Model Points.txt
Total number of words in a file are =  403098
Processing chunk  0
Remaining words =  397978
Remaining words =  397978
Processing chunk  5120
Remaining words =  387738
Remaining words =  387738


Based on the information provided in the Excel tab, I can identify the following tables present in the Calculation Engine tab:

1. Year Reference Table:
   - Located in column A, starting from cell A11 and continuing down to A1010
   - Contains a series of years, with each subsequent year incrementing by 1
   - Formula used: =A[previous row]+1

2. Policy Type Indicator Table:
   - Located in column B, starting from cell B11 and continuing down to B1010
   - Uses an IF statement to determine whether a policy is In-Force (IF) or New Business (NB) based on the year
   - Formula used: =IF(A[row]>$B$2,"-",IF(A[row]<=$B$3,$A$3,$A$4))

3. Reference Data Table:
   - Located in the upper part of the sheet, likely cells A1:B4 and possibly extending to column G
   - Contains reference values for:
     - Policy Count (cell B2)
     - In-Force (IF) indicator (cell A3)
     - New Business (NB) indicator (cell A4)
     - Other reference values in cells B3 and E2:G7 (exact contents not provided)

These tables form the core structure of a typical actuarial projection model, where:
- The Year Reference Table provides the time dimension for the projection
- The Policy Type Indicator Table distinguishes between existing (in-force) policies and new business
- The Reference Data Table likely contains key assumptions or parameters used in the calculations

The model appears to be set up to project policy counts or other metrics over a long time horizon (1000 years), which is common in long-term insurance or pension valuations. The structure allows for separate treatment of in-force business and new business, which is crucial for many actuarial analyses and financial projections in the insurance industry.Based on the provided Excel formulas in column C from rows 221 to 476, I can identify one main table or variable in the Calculation Engine tab:

Policy Classification Table:

This appears to be a large table that classifies policies into different categories based on certain criteria. The formula is consistent across all cells and uses nested IF statements to determine the policy type. Here's a breakdown of the logic:

1. If the previous cell is "-", the current cell is also "-".
2. If the value in column A is greater than or equal to $B$2 + 1, the result is "-".
3. If the value in column A is less than or equal to $E$2, the result is "IF Policies".
4. If the value in column A falls between $E$2 and $G$3, $G$4, $G$5, $G$6, or $G$7, the result is "NB Sales Yr" followed by a number (likely representing different years of new business sales).

This table is likely used to categorize policies for actuarial calculations, possibly for determining reserves, premiums, or other policy-related metrics. The classification appears to be based on policy dates or durations (represented by values in column A) and uses various threshold values stored in cells B2, E2, G3, G4, G5, G6, and G7.

The table is a model variable since it contains formulas rather than just input values. It's designed to automatically classify policies based on predefined criteria, which is a common practice in actuarial models for policy administration and valuation.